# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform"""
    #clear colors
    color_select = np.copy(img)
    #threshold for yellow and white lanes
    red_threshold = 180
    green_threshold = 180
    blue_threshold = 0
    rgb_threshold = [red_threshold, green_threshold, blue_threshold]
    thresholds = (img[:,:,0] < rgb_threshold[0]) \
            | (img[:,:,1] < rgb_threshold[1]) \
            | (img[:,:,2] < rgb_threshold[2])
    color_select[thresholds] = [0,0,0]
    #get gray image from masked image
    return cv2.cvtColor( color_select, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    #Sometimes, the parameters used by the hough Transform may result into 
    #an empty set of lines. This line avoids the program to break during
    #running time.
    if lines is None:
        return
    
    #compute the slope of the line
    def compute_slope(x1,y1,x2,y2):
        #check for non valid division
        if x2 - x1 == 0:
            return None
        return ((y2-y1)/(x2-x1))
    
    #left and right vertices 
    left = (-1,-1,-1,-1)  
    right = (-1,-1,-1,-1)  
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            slope = compute_slope(x1,y1,x2,y2)#((y2-y1)/(x2-x1))    
            if slope is None:
                #ignore if slope is not valid
                continue
                
            if y1 < 290 or y2 < 290:
                #ignore point, outside of lane interesting area
                continue 
                
            if slope < 0:     
                #ignore points of right side
                if x1 > img.shape[1]/2 or x2 > img.shape[1]/2:
                    #ignore point, outside of lane interesting area
                    continue                
                #left side
                (lx1, ly1, lx2, ly2) = left
                #get max and min vertices for left
                if lx1==-1:
                    left = (x1,y1,x2,y2)
                else:
                    (xx1, yy1, xx2, yy2) = left
                    if x1 < lx1:
                        xx1 = x1
                    if y1 > ly1:
                        yy1 = y1
                    if x2 > lx2:
                        xx2 = x2
                    if y2 < ly2:
                        yy2 = y2
                    left = (xx1, yy1, xx2, yy2)
            else: 
                if x1 < img.shape[1]/2 or x2 < img.shape[1]/2:
                    #ignore point, outside of lane interesting area
                    continue  
                #rigth side
                (rx1, ry1, rx2, ry2) = right
                #get max and min vertices for right
                if rx1==-1:
                    right = (x1,y1,x2,y2)
                else:
                    (xx1, yy1, xx2, yy2) = right
                    if x1 < rx1:
                        xx1 = x1
                    if y1 < ry1:
                        yy1 = y1
                    if x2 > rx2:
                        xx2 = x2
                    if y2 > ry2:
                        yy2 = y2
                    right = (xx1, yy1, xx2, yy2)  
    
    #add left line
    (lx1, ly1, lx2, ly2) = left
    left_slope = compute_slope(lx1, ly1, lx2, ly2) 
    if left_slope is not None:
        left_constante = ly1 - left_slope*lx1
        #compute new left vertices
        by = img.shape[1]
        bx = int((by - left_constante)/left_slope)
        uy = ly2
        ux = int((uy - left_constante)/left_slope)
        #draw left line
        cv2.line(img, (bx, by), (ux, uy), color, thickness)
    
    #add right line
    (rx1, ry1, rx2, ry2) = right
    right_slope = compute_slope(rx1, ry1, rx2, ry2)
    if right_slope is not None:
        right_constante = ry1 - right_slope*rx1
        #compute new right vertices
        by = img.shape[1]
        bx = int((by - right_constante)/right_slope)
        uy = ry1
        ux = int((uy - right_constante)/right_slope)
        #draw right line
        cv2.line(img, (bx, by), (ux, uy), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
import os
#Image folder
PATH = "test_images/"

# image pipeline to identify lanes
def add_lines(img):
    
    #image features
    imshape = img.shape
    #get a gray scale version of the image
    gray = grayscale(img)
    #apply gaussian smootthing 
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)
    #apply candy transform (gradients)
    low_threshold = 80
    high_threshold = 100
    edges = canny(blur_gray, low_threshold, high_threshold)
    #apply mask to remove non interest area
    vertices = np.array([[(0,imshape[0]),(420,320),(510, 320),(imshape[1], imshape[0])]], dtype=np.int32)
    masked = region_of_interest(edges, vertices)
    #apply Hough Transform and draw the lines
    rho = 2
    theta = 1*(np.pi/180)
    threshold = 35
    min_line_len = 30
    max_line_gap = 50
    line_image = hough_lines(masked, rho, theta, threshold, min_line_len, max_line_gap)
    #create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges))
    line_edges = weighted_img(line_image, color_edges)
    #show image for debug
    #plt.imshow(line_edges)  
    return line_image

for f in os.listdir(PATH): 
    #check if it is a copy or an original image 
    if f.find('copy') != -1 or f.find('part') != -1:
        #ignore, copy image
        continue
    #open image 
    img = mpimg.imread(PATH + f)    
    #identify and draw lines
    img_lines = add_lines(img)
    #save image
    result = weighted_img(img_lines, img)
    mpimg.imsave(PATH + "copy_" + f, result)
    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    #add lines to image
    black_image_with_lines = add_lines(image)
    #stack it over the original image
    result = weighted_img(black_image_with_lines, image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))